# Data Preparation

# Inhouds opgaven  data preparation

## 1. Libraries
## 2. Data Import
## 3. Data retrieve
## 4. Kolommen selecteren
## 5. Duplicaten verwijderen
## 6. Na waardes
## 7. Dtypes omzetten
## 8. Prognose aanpasing
## 9. Duur kolommen aanmaken en aanpassen
## 10. Target encoding


## 1. Libraries

Hier importeren we de benodigde libraries voor het aanpassen van de data voor de modellen.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from category_encoders import LeaveOneOutEncoder

## 2. Data import

We laden de data van het csv bestand in zonder de Nvt

In [2]:
cols = list(pd.read_csv("sap_storing_data_hu_project.csv", nrows =1))
exclusion_list = ['stm_sap_mon_meld_ddt', 'stm_mon_begin_ddt','stm_mon_toelichting_trdl','stm_oh_pg_mld', 'stm_scenario_mon','stm_mon_nr_status_omschr', 'stm_mon_nr__statuscode', 'stm_mon_nr_status_wijzdd', 'stm_objectdl_code_gst', 'stm_objectdl_groep_gst','stm_progfh_in_ddt','stm_progfh_in_invoer_ddt','stm_progfh_gw_ddt','stm_progfh_gw_lwd_ddt','stm_progfh_hz','stm_veroorz_groep','stm_veroorz_code','stm_veroorz_tekst_kort','stm_effect', 'stm_afspr_aanvangddt','stm_mon_eind_ddt','stm_mon_vhdsincident','stm_dir_betrok_tr','stm_aangelegd_dd','stm_aangelegd_tijd','stm_mon_begindatum','stm_mon_begintijd','stm_progfh_gw_datum','stm_progfh_gw_lwd_datum','stm_progfh_gw_lwd_tijd','stm_progfh_gw_duur','stm_afspr_aanvangdd','stm_afspr_aanvangtijd','stm_afspr_aanvangtijd','stm_mon_eind_tijd','stm_controle_dd','stm_akkoord_mon_toewijz','stm_status_sapnaarmon','stm_fact_jn','stm_akkoord_melding_jn','stm_afsluit_ddt','stm_afsluit_dd','stm_afsluit_tijd','stm_rec_toegev_ddt','stm_hinderwaarde','stm_actie','stm_standplaats','stm_status_gebr','stm_wbi_nummer','stm_projnr','stm_historie_toelichting','stm_schade_verhaalb_jn','stm_schadenr','stm_schade_status_ga','stm_schade_statusdatum','stm_relatiervo_vorig','stm_relatiervo_volgend','stm_relatiervo','stm_afspr_func_hersteldd','stm_afspr_func_hersteltijd','stm_sorteerveld','stm_rapportage_maand','stm_rapportage_jaar','stm_x_bron_publ_dt','stm_x_bron_bestandsnaam','stm_x_bron_arch_dt','stm_x_actueel_ind','stm_x_run_id','stm_x_bk','stm_x_start_sessie_dt','stm_x_vervallen_ind']
table = pd.read_csv('sap_storing_data_hu_project.csv', low_memory=False, usecols=[i for i in cols if i not in exclusion_list])

## 3. Data retrieve

Functie voor het ophalen van de data voor modeling.

In [3]:
def return_model_data():
    return [table, equipm_soort_encoder]

## 4. Kolommen selecteren

Hier onder worden bepaalde kolommen verwijdert uit de dataset voor memory gebruik en daarnaast ook de relevantie voor het model. Er wordt in business understanding duidelijker uitgelegd waarom we deze kolommen verwijderen en of aanpassen.

We verwijderen eerst de kolommen die geen goede data bevatten.

In [4]:
table = table.drop(['stm_sap_meldtekst_lang', 'stm_sap_meldtekst', 'stm_oorz_tkst', 'stm_oorz_tekst_kort', 'stm_equipm_omschr_mld', 'stm_equipm_omschr_gst', 'Unnamed: 0', 'stm_geo_gst_uit_functiepl', 'stm_functiepl_gst', 'stm_arbeid', 'stm_evb'], axis=1)

Daarna verwijderen we de kolommen die duplicaat informatie bevat.

In [5]:
table = table.drop(['stm_sap_melddatum', 'stm_sap_storeind_ddt', 'stm_sap_meldtijd', 'stm_aanngeb_tijd', 'stm_aanngeb_dd', 'stm_aanntpl_dd', 'stm_aanntpl_tijd', 'stm_progfh_in_datum', 'stm_progfh_in_tijd', 'stm_progfh_in_invoer_dat', 'stm_progfh_in_invoer_tijd', 'stm_progfh_gw_tijd', 'stm_fh_dd', 'stm_fh_tijd', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd', 'stm_mon_eind_datum'], axis=1)

We verwijderen vervolgens de kolommen waar te veel na's in staan of te weinig informatie bevatten.

In [6]:
table = table.drop(['stm_tao_indicator_vorige', 'stm_pplg_naar', 'stm_tao_soort_mutatie', 'stm_pplg_van', 'stm_dstrglp_van', 'stm_dstrglp_naar'], axis=1)

Hier na verwijderen we de kolommen die informatie bevat die inrelevant is voor het model

In [7]:
table = table.drop(['stm_vl_post', 'stm_mon_nr'], axis=1)

Er wordt hieronder op basis van de business understanding verschillende functie herstel duur data verwijdert. Dit wordt in data understanding uitgelegd.

In [8]:
table = table[table['stm_fh_duur'] > 0]

In [9]:
table = table[table['stm_fh_duur'] <= 600]

## 5. Duplicaten verwijderen

De rijen in de data bevatten duplicaten van dezelfde melding. Die meldingen worden hier verwijdert op de last occurence omdat het laatste item de laatste verandering is in de melding

In [10]:
table[table.duplicated('#stm_sap_meldnr', keep = False)].sort_values(by=['#stm_sap_meldnr']).head(4)

,#stm_sap_meldnr,stm_sap_meld_ddt,stm_geo_mld,stm_geo_mld_uit_functiepl,stm_equipm_nr_mld,stm_equipm_soort_mld,stm_km_van_mld,stm_km_tot_mld,stm_prioriteit,stm_status_melding_sap,...,stm_tao_beinvloedbaar_indicator,stm_contractgeb_mld,stm_functiepl_mld,stm_techn_mld,stm_contractgeb_gst,stm_techn_gst,stm_progfh_in_duur,stm_progfh_gw_teller,stm_fh_duur,stm_reactie_duur
35216,50117666,10/01/2013 10:42:00,541.0,541.0,NaN,NaN,13100.0,13200.0,9.0,MAFS,...,B,NaN,541,NaN,11.0,S,99999999,0.0,3.0,99999999.0
35217,50117666,10/01/2013 10:42:00,541.0,541.0,NaN,NaN,13100.0,13200.0,9.0,MAFS,...,B,NaN,541,NaN,11.0,S,99999999,0.0,3.0,99999999.0
35407,50117856,15/01/2013 15:51:00,507.0,507.0,10201352.0,WISSEL,93639.0,0.0,9.0,MAFS,...,B,30.0,507-302-1000072761,B,30.0,B,99999999,0.0,25.0,99999999.0
35408,50117856,15/01/2013 15:51:00,507.0,507.0,10201352.0,WISSEL,93639.0,0.0,9.0,MAFS,...,B,30.0,507-302-1000072761,B,30.0,B,99999999,0.0,25.0,99999999.0


In [11]:
table.drop_duplicates(subset=['#stm_sap_meldnr'],inplace=True, keep='last')
print(table.shape)

(374871, 34)


## 6. Na waardes

Vervolgens veranderen we de Na waardes van de kolommen die gebruikt gaan worden naar None. Hier wordt dieper op in gegaan in business understanding.

In [12]:
table['stm_equipm_soort_gst'] = table['stm_equipm_soort_gst'].fillna('None')
table['stm_equipm_nr_gst'] = table['stm_equipm_nr_gst'].fillna('None')
table['stm_geo_gst'] = table['stm_geo_gst'].fillna('None')
table['stm_contractgeb_mld'] = table['stm_contractgeb_mld'].fillna('None')
table['stm_equipm_soort_mld'] = table['stm_equipm_soort_mld'].fillna('None')
table['stm_equipm_nr_mld'] = table['stm_equipm_nr_mld'].fillna('None')
table['stm_techn_mld'] = table['stm_techn_mld'].fillna('None')
table['stm_equipm_nr_mld'] = table['stm_equipm_nr_mld'].fillna(0)

## 7. Dtypes omzetten

Hier onder veranderen we de kolommen naar de bijbehorende types voor de modellen.

In [13]:
table['stm_aanntpl_ddt'] = pd.to_datetime(table['stm_aanntpl_ddt'], errors='coerce', dayfirst=False, infer_datetime_format=True)
table['stm_aanngeb_ddt'] = pd.to_datetime(table['stm_aanngeb_ddt'], errors='coerce', dayfirst=True, infer_datetime_format=True)
table['stm_sap_meld_ddt'] = pd.to_datetime(table['stm_sap_meld_ddt'], errors='coerce', dayfirst=True,infer_datetime_format=True)

table['stm_aanntpl_ddt'] = table['stm_aanntpl_ddt'].fillna(table['stm_sap_meld_ddt'])

table['stm_fh_duur'] =  pd.to_numeric(table['stm_fh_duur'], errors='coerce')
table['stm_progfh_in_duur'] =  pd.to_numeric(table['stm_progfh_in_duur'], errors='coerce')
table['stm_equipm_soort_mld'] = table['stm_equipm_soort_mld'].astype('category')
table['stm_equipm_nr_mld'] = table['stm_equipm_nr_mld'].astype('category')


## 8. Prognose aanpasing

Voor de verschillende prognose duur word er hier prognose die ver afliggen van de echte functie herstel duur verwijdert.

In [14]:
 table = table[table['stm_progfh_in_duur'] <= 900]

## 9. Duur kolommen aanmaken en aanpassen

Door de datetime kolommen kan der een duur bepaald worden tussen tijden. Hier onder wordt die tijd aan gemaakt en aangepast als er foutieve data in voorkomt

In [15]:
table = table[table['stm_progfh_in_duur'] <= 900]
table['stm_meld_gebeld_duur'] = (table['stm_aanngeb_ddt'] - table['stm_sap_meld_ddt']).astype('timedelta64[m]')
table['stm_meld_ter_plekken_duur'] = (table['stm_aanntpl_ddt'] - table['stm_sap_meld_ddt']).astype('timedelta64[m]')

# # van af gebeld tot ter plaatsen
table['stm_gebeld_ter_plekken_duur'] = (table['stm_aanntpl_ddt'] - table['stm_aanngeb_ddt']).astype('timedelta64[m]')


table.loc[(table['stm_meld_gebeld_duur'] < 0), 'stm_meld_gebeld_duur'] = 0
table.loc[(table['stm_meld_ter_plekken_duur'] < 0), 'stm_meld_ter_plekken_duur'] = 0
table.loc[(table['stm_gebeld_ter_plekken_duur'] < 0), 'stm_gebeld_ter_plekken_duur'] = 0

table['stm_meld_gebeld_duur'] = table['stm_meld_gebeld_duur'].fillna(0)
table['stm_meld_ter_plekken_duur'] = table['stm_meld_ter_plekken_duur'].fillna(0)
table['stm_gebeld_ter_plekken_duur'] = table['stm_gebeld_ter_plekken_duur'].fillna(0)

table = table[table['stm_gebeld_ter_plekken_duur'] < 1000]

value_counts = table['stm_equipm_soort_mld'].value_counts()  
table['stm_equipm_soort_mld'].replace((value_counts[value_counts <= 1].index), np.nan, inplace=True)

## 10. Target encoding

Voor de kolommen gekozen zijn als feature worden hier twee kolommen omgezet naar de leave one out encoder. In business understanding wordt dit verder uitgelegd.

In [16]:
equipm_soort_encoder = LeaveOneOutEncoder(return_df=True, sigma=0.05)
table['stm_equipm_soort_mld_looe_mean'] = equipm_soort_encoder.fit_transform(table['stm_equipm_soort_mld'].astype('category'), table['stm_fh_duur'])

tech_soort_encoder = LeaveOneOutEncoder(return_df=True, sigma=0.05)
table['stm_techn_mld_looe_mean'] = tech_soort_encoder.fit_transform(table['stm_techn_mld'].astype('category'), table['stm_fh_duur'])

In [17]:
table[['stm_techn_mld_looe_mean', 'stm_techn_mld_looe_mean']]

,stm_techn_mld_looe_mean,stm_techn_mld_looe_mean
32099,149.071866,149.071866
52801,117.020242,117.020242
55827,135.856884,135.856884
84561,124.189472,124.189472
86473,133.377696,133.377696
...,...,...
898509,124.592164,124.592164
898519,131.795680,131.795680
898521,127.847297,127.847297
898523,125.264880,125.264880
